## 4. Preparing to use image services...


In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import requests, io, astropy
from IPython.display import Image, display

## For handling ordinary astropy Tables
from astropy.table import Table, vstack

## For reading FITS files
import astropy.io.fits as apfits

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## our stuff
import sys
# Use the NASA_NAVO/astroquery
from navo_utils.cone import Cone
from navo_utils.registry import Registry
from navo_utils.registry import display_results
from navo_utils.image import Image
import astroquery

First we need to know what registry services have the image data that we want. The Registry query function is useful here for picking out the "image" (SIA) service. Here, we define a shortcut to query the registry and output a list of the MAST data archives that contain ultraviolet wavelength data. However, you can easily select optical infrared, or any other wavelengths by modifying the "waveband" parameter or select irsa, heasarc, or leave the "source" field blank if you want other data archives. 

This step is like deciding which website to go to to access your image data. But this method will give you greater flexibility. If you know the access_url for your data, you can skip this step. 

In [2]:
##### THIS WORKS... but trying ImageClass method... 
#def list_image_services(**kwargs):
#    return Registry.query(service_type="image",**kwargs)
#results=list_image_services(waveband="uv", source="mast") # THIS WORKS
#
#print('Found {} results:'.format(len(results)))
#
#print(results.columns)
#ivoid=results[:]['ivoid']
#url=results[:]['access_url']
#
# For visual reference:
#for i in range(len(url)):
#    print(i, ':', ivoid[i], '; ', url[i])
#    
##### NEW STUFF ### Streamlining... Don't need this step! 



## 5 SIA

We find the relevant image data within catalogs containing image data. 

### 5.1 Using a specific data catalog to search for image data for a source
Once you find which data archive contains data that you are interested in, you can find available image data from an SIA service using the access_url. 

For example, say that we know the CANDELS data is best for our purpose. This corresponds to the number 1. Here we show how you get a table with information about images within this data catalog for sources within 5 arcsec of the given ra,dec position. 

In [3]:
### NOTES: TROUBLE-SHOOTING in Progress... 
# 1. Works for example above: results=list_image_services(waveband="uv", source="mast")
# 2. With X-ray: results=list_image_services(waveband="x-ray", source="heasarc"), choose #6 (chanmaster catalog) get following error:
#   ERROR parsing response as astropy Table: looks like the content isn't the expected VO table XML? Returning an empty table. Look at its meta data to debug.
#   <TableColumns names=()>
#   Found 0 results:
# 3. With IRSA: results=list_image_services(waveband="infrared", source="irsa") # THIS breaks things... 
#   Choose #49 (S-CANDELS) get same error as above: 
#   ERROR parsing response as astropy Table: looks like the content isn't the expected VO table XML? Returning an empty table. Look at its meta data to debug.
#   <TableColumns names=()>
#   Found 0 results:
# Also Note: for "waveband" need to use "infrared" not "IR" -- how does a user know this?!
# Think escape characters in url is screwing things up... e.g., "&amp;" "%5C", etc.  
# Got this to work via brute force... 
# im_info=sia_query(access_url='https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=S-CANDELS&hdr_location=\S-CANDELSDataPath\&SIAP_ACTIVE=1&collection_desc=S-CANDELS&', ra='189.22917', dec='62.23830', size='.001')
# print(im_info.columns)
# print()
# print('Found {} results:'.format(len(im_info)))
# print()
# for i in range(len(im_info)):
#    print(im_info[i]['col1'],  im_info[i]['col2'], im_info[i]['col3'], im_info[i]['col4'])
###### END NOTES: 


In [4]:
### This works ... Moving on... 
#im_info=sia_query(access_url=results[1]['access_url'], ra='189.22917', dec='62.23830', size='.001') 
   
    # For visual reference:
#print(im_info.columns)
#print()
#print('Found {} results:'.format(len(im_info)))
#print()
#for i in range(len(im_info)):
#  print(im_info[i]['ra_j2000'], im_info[i]['dec_j2000'],  im_info[i]['bandpass_id'], im_info[i]['url'])


http://archive.stsci.edu/siap/search.php?id=candels&amp;
Found 1 services to query.
    Querying service http://archive.stsci.edu/siap/search.php?id=candels&
    Got 6 results for source number 0


In [7]:
############## USING NEW METHOD -- IMAGE CLASS #######
results=Registry.query(service_type="image",waveband="uv", source="mast")
print('Found {} results:'.format(len(results)))

print(results.columns)
ivoid=results[:]['ivoid']
url=results[:]['access_url']
#
# For visual reference:
for i in range(len(url)):
    print(i, ':', ivoid[i], '; ', url[i])
print(results[1]['access_url'])
im_info=Image.query(coords='189.22917,62.23830', inradius='.001', services=results[1])

http://archive.stsci.edu/siap/search.php?id=candels&amp;
Found 1 services to query.
    Querying service http://archive.stsci.edu/siap/search.php?id=candels&
    Got 6 results for source number 0
Found 1 services to query.
    Querying service http://archive.stsci.edu/siap/search.php?id=candels&
    Got 6 results for source number 0


### 5.2 Searching for image data for a given source in all available catalogs
Perhaps you don't know which catalog has the image that you want, so you get a table list for all images that contain your given source. 

The Image class was constructed to do this for you. The example below shows how to do this for all the services, still setting the waveband and data source fields, which one can easily modify to suit their specific example. 
 <font color='red'>[THIS IS NOT WORKING... MIGHT BE BECAUSE OF SIMILAR ISSUE AS ABOVE!]</font> 


In [5]:
im_info=Image.query(coords='189.22917,62.23830', inradius='.001', waveband='uv', source='mast', max_services=20)
### Will try to get only uv, etc, next ... 

Found 12 services to query.
    Querying service https://irsa.ipac.caltech.edu/cgi-bin/Atlas/nph-atlas?mission=Scrapbook&hdr_location=%5CScrapbookDataPath%5C&collection_desc=The+MAST+Image%2FSpectra+Scrapbook+%28Scrapbook%29&SIAP_ACTIVE=1&
    Got 240 results for source number 0
    Querying service http://archive.stsci.edu/siap/search.php?id=candels&
    Got 6 results for source number 0
    Querying service http://archive.stsci.edu/siap/search.php?id=maoz_atlas&
    (Got no results for source number 0)
    Querying service http://archive.stsci.edu/siap/search.php?id=UIT&
    (Got no results for source number 0)
    Querying service http://archive.stsci.edu/stpr/vo_search.php?
    (Got no results for source number 0)
    Querying service http://archive.stsci.edu/siap/search.php?id=galex_atlas&
    (Got no results for source number 0)
    Querying service http://archive.stsci.edu/siap/search.php?id=udfuv&
    (Got no results for source number 0)
    Querying service http://archive.stsc

## 6 Querying a list of sources

We want to be able to give a list of sources and get back something useful  for matches/non-matches. The image query function can also handle lists of coordinates.   <font color='red'>[SHOW EXAMPLE HERE]</font> 

## 7-8. Retrieving images

You can look at the images_info and pick one to download:

    Image.get( images_info[6], filename='my_file.fits')
    
or get the image data to hand to the plotter:

    image=Image.get( images_info[6] )
    plt.imshow( image,  cmap='gray', origin='lower',vmax=0.02 )
    
where it downloads it to filename if specified, or to a temporary filename if not and then reads it in and returns the image data if not.

Or hand a list and return a list of images:

    images=Image.get(images_info )
    plt.imshow( images[0],  cmap='gray', origin='lower',vmax=0.02 )
    
<font color='red'>[SHOW EXAMPLE HERE]</font> 